In [1]:
import os 
import numpy as np
import keras
from keras.models import load_model

In [2]:
# scikit-learn libraries
from sklearn.metrics import roc_auc_score, confusion_matrix

In [3]:
# Define paths
base_path = os.path.abspath("../../../")
dataset_path = os.path.join(base_path, "datasets/ISIC-2016/baseline", "isic2016numpy")
print("Contents in folder:", os.listdir(dataset_path))

Contents in folder: ['x_test.npy', 'x_train.npy', 'y_test.npy', 'y_train.npy']


In [4]:
# Load test data
x_test = np.load("{}/x_test.npy".format(dataset_path))
y_test = np.load("{}/y_test.npy".format(dataset_path))

# Standardize
x_test = x_test.astype('float32')
x_test /= 255
assert min(x_test.flatten()) == 0.0, "Not preprocessed"

# Show shape
print("Dataset sample size :", x_test.shape, y_test.shape)

Dataset sample size : (379, 256, 256, 3) (379, 2)


In [5]:
# Model path
weights_path = "{}/{}.h5".format(os.path.join(base_path, "models/ISIC-2016/baseline"), "MelaNet")
model = load_model(weights_path, compile = False) 
print("Model loaded")

Model loaded


In [6]:
# Make predictions using trained model
y_pred = model.predict(x_test, verbose=1)
print("Predictions: ", y_pred.shape)

12/12 [==============================] - 12s 463ms/step
Predictions:  (379, 2)


In [7]:
# Convert ground truth to column values
y_test_flat = np.argmax(y_test, axis=1)
print("After flattening ground truth: ", y_test_flat.shape)

After flattening ground truth:  (379,)


In [8]:
# Get labels from predictions
y_pred_flat = np.array([np.argmax(pred) for pred in y_pred]) # y_pred[1] -> probability for class 1 
print("Binarize probability values: ", y_pred_flat.shape)

Binarize probability values:  (379,)


In [9]:
# Sanity check
print(y_test.shape, y_test_flat.shape, y_pred.shape, y_pred_flat.shape)

(379, 2) (379,) (379, 2) (379,)


# AUC

In [10]:
print("MelaNet | AUC: ", '{0:.2f}%'.format(roc_auc_score(y_test, y_pred)*100))

MelaNet | AUC:  81.19%


# Sensitivity & Specificity

In [11]:
TN, FP, FN, TP = confusion_matrix(y_test_flat, y_pred_flat).ravel()
print("MelaNet | Sensitivity :", '{0:.2f}%'.format(TP/(TP+FN)*100))
print("MelaNet | Specificity :", '{0:.2f}%'.format(TN/(TN+FP)*100))

MelaNet | Sensitivity : 70.67%
MelaNet | Specificity : 80.59%
